In [1]:
!pip install selenium
!pip3 install webdriver_manager

In [4]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time

def scrape_companies(url):
    firefox_options = Options()
    # firefox_options.add_argument('--headless')  # Uncomment this line if you want to run in headless mode
    firefox_options.add_argument('--no-sandbox')
    firefox_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)
    driver.get(url)

    SCROLL_PAUSE_TIME = 20
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "https://twitter.com/SECGov")))

    html = driver.page_source
    soup = bs(html, 'html.parser')

    relevant = soup.find_all('tr', class_='pr-list-page-row')

    names = []
    for li in relevant:
        for html_class in li.find_all('div', class_="release-view__respondents"):
            org = html_class.find('a').get_text()
            names.append(org)

    driver.quit()
    return names

# List of URLs to scrape
urls = [
    'https://www.sec.gov/litigation/litreleases?aId=&populate=&year=2023&month=All',
    'https://www.sec.gov/litigation/litreleases?aId=&populate=&year=2023&month=All&page=1',
    'https://www.sec.gov/litigation/litreleases?aId=&populate=&year=2023&month=All&page=2',
    'https://www.sec.gov/litigation/litreleases?aId=&populate=&year=2023&month=All&page=3'
    
    # Add more URLs here
]

# Iterate over each URL, scrape the product links, and store them in product_links
all_names = []
for url in urls:
    names = scrape_companies(url)
    all_names.extend(names)

# Print the first 10 product links
print(all_names[:10])


C:\Users\avish\AppData\Local\Temp\ipykernel_20240\2944475637.py:17: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)


['Diana Mae Fernandez', 'Jonathan Larmore, et al.', 'Laura Tyler Perryman', 'Dozy Mmobuosi, et al.', 'Agridime LLC, Joshua Link, and Jed Wood', 'Dozy Mmobuosi, et al.', 'Titanium Capital LLC; Henry Abdo; Carol Ann Barsh', 'Justin Murphy and Mara Investments LLC', 'David Blaszczak, et al.', 'GPB Capital Holdings LLC']


In [5]:
import csv

# Open the file in binary mode with UTF-8 encoding
with open("SEC_litigation.csv", "w", newline="", encoding="utf-8") as Output_File:
    writer = csv.writer(Output_File)
    writer.writerow(["Companies" ])

    for x in range(0, len(all_names)):
        writer.writerow([all_names[x]])
